In [2]:
#Task 1
import torch
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import random_split

transform = transforms.Compose([
    transforms.ToTensor()
])

dataset = datasets.MNIST(
    root="./data",
    train=True,
    download=True,
    transform=transform
)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [3]:
#Task 2
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

test_dataset = datasets.MNIST(
    root="./data",
    train=False,
    download=True,
    transform=transform
)

test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [5]:
#Task 3
import torch.nn as nn
import torch.optim as optim

class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = NeuralNet()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 5

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss:.4f}")

Epoch [1/5], Loss: 284.5314
Epoch [2/5], Loss: 119.3879
Epoch [3/5], Loss: 82.2203
Epoch [4/5], Loss: 61.8604
Epoch [5/5], Loss: 48.9049


In [6]:
#Task 4
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.numpy())
        all_labels.extend(labels.numpy())

accuracy = accuracy_score(all_labels, all_preds)
print("Accuracy:", accuracy)

print("Classification Report:\n")
print(classification_report(all_labels, all_preds))

print("Confusion Matrix:\n")
print(confusion_matrix(all_labels, all_preds))

Accuracy: 0.9689
Classification Report:

              precision    recall  f1-score   support

           0       0.98      0.99      0.98       980
           1       0.97      0.99      0.98      1135
           2       0.97      0.97      0.97      1032
           3       0.97      0.98      0.97      1010
           4       0.98      0.95      0.97       982
           5       0.95      0.97      0.96       892
           6       0.96      0.98      0.97       958
           7       0.94      0.98      0.96      1028
           8       0.97      0.94      0.96       974
           9       0.98      0.93      0.95      1009

    accuracy                           0.97     10000
   macro avg       0.97      0.97      0.97     10000
weighted avg       0.97      0.97      0.97     10000

Confusion Matrix:

[[ 968    0    1    1    0    1    5    2    1    1]
 [   0 1126    3    1    0    1    3    0    1    0]
 [   5    4 1003    2    2    0    3   10    3    0]
 [   0    1    4  986 